In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline

from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
import re
from glob import glob
from IPython.display import Image

import seaborn as sns
import folium as g

import matplotlib.pyplot as plt
from matplotlib import rc
# 한글 폰트 문제 해결
rc('font', family='AppleGothic')
# 차트 축 <- 음수 부호 지원
plt.rcParams['axes.unicode_minus'] = Falseb

In [9]:
import sklearn
from sklearn.datasets import load_iris, load_breast_cancer

from sklearn.model_selection import train_test_split,KFold,StratifiedKFold , cross_val_score, cross_validate,GridSearchCV
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import missingno as msno

#### 정확도 : 실 데이터와 예측 데이터가 얼마나 같은지를 판단하는 자료

- 문제점 : 이진분류의 경우 모델의 성능을 왜곡할수 있다
- why?:
- solution : f1-score

#### 분류모델 성능평가를 위한 이해 

In [8]:
target = [1,0,1,1,1,0,1,0,0,0,1]
predict = [1,1,1,1,0,0,0,0,1,1,1]

tp = tn = fn = fp = 0
for i in range(len(target)):
    # TP
    if target[i]==1 and predict[i]==1:
        tp+=1
    # TN
    elif target[i]==0 and predict[i]==0:
        tn+=1
    # FN
    elif target[i]==1 and predict[i]==0:
        fn+=1
    # FP
    elif target[i]==0 and predict[i]==1:
        fp+=1
        
print('TP:',tp)
print('TN:',tn)
print('FN:',fn)
print('FP:',fp)

TP: 4
TN: 2
FN: 2
FP: 3


In [10]:
confusion_matrix(target,predict)

array([[2, 3],
       [2, 4]])

In [12]:
accuracy_score(target,predict)

0.5454545454545454

In [13]:
precision_score(target,predict)

0.5714285714285714

In [14]:
recall_score(target,predict)

0.6666666666666666

In [11]:
f1_score(target,predict)

0.6153846153846153

In [ ]:
# 이진분류
# 제품을 생상하는 제조공장에서 품질 테스트를 실시하여 불량품을 찾아내고 불량품을 공장으로 돌려보낸다

# 품질테스스 결과가 양성 ->불량품을 예측 음성->정상제품으로 예측

# TP : 불량품이라고 예측한것중 실제 불량품인것
# TN : 정상이라고 에측한것중 실제 정상인것
# 